# CNMa
###### Correlation N Matching

In [1]:
#import IP_address_filter
#import SG_IP_matching
#import xml_to_csv
import VT_API
import OS_API
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os
from tqdm import tqdm
import json

## Search in OpenSearch

In [3]:
#columns of interest
COLS = ['peerIP',
        'peerPort',
        'hostIP',
        'hostPort',
        'commands',
        'hashes',
        'urls',
        'loggedin',
        'startTime',
        'endTime',
        'sort_num',
        'peerCountry',
        'hostCountry'
       ]

SIZE = 10000

In [4]:
def yesnocheck(param):
    while True:
        # Note: Python 2.x users should use raw_input, the equivalent of 3.x's input
        ans = input(f"{param}")
        if ans not in ['y', 'n']:
            continue            
        else:
            break
    return ans

#ans = yesnocheck("test")
#ans

In [5]:
#query inputting

defaultquery = [
    {'range': {'startTime':{'gte': 'now-1w/w',
                            'lte': 'now',
                            'format': 'strict_date_optional_time'}}},
                 {'exists': {'field': 'hashes'}},
                 {'match': {'geoip.country_name': 'Singapore'}},
                 {'query_string': {'default_field': 'commands', 'query': 'wget OR tftp'}}
               ]


def craft_query(devmode):
    queryls = []
    #dev mode to return default eg.
    while devmode:
        return defaultquery
    
    #custom input
    #Time range
    ans = yesnocheck("Filter on Time Range? y/n")
    if ans == 'y':
        start = str(input("Input start date:"))
        end = str(input("Input end date:"))
        rnge = {"range": {"startTime": {"gte": start, "lte": end, "format": "strict_date_optional_time"}}}
        queryls.append(rnge)    

    #Existing field
    while True:
        ans = yesnocheck("Filter if a field Exists? y/n")
        #print(ans)
        if ans == 'y':
            exist = input("Please enter field:")
            exst = {'exists': {'field': exist}}
            queryls.append(exst)
        else:
            break
    
    #Exact match on field
    while True:
        ans = yesnocheck("Exact match on a field? y/n")
        #print(ans)
        if ans == 'y':
            field = input("Please enter field:")
            value = input("Enter value to match:")
            mtch = {"match": {field: value}}
            queryls.append(mtch)
        else:
            break
    
    #General match query
    while True:
        ans = yesnocheck("General search query on a field? y/n")
        #print(ans)
        if ans == 'y':
            field = input("Please enter field:")
            query = input("Enter query:")
            qry = {"query_string": {"default_field": field, "query": query}}
            queryls.append(qry)
        else:
            break
    
    return queryls
    
#queryls = craft_query(True)
#queryls

In [6]:
size = 10000
search_after = [0]
df_os = pd.DataFrame(columns=COLS)

queryls = craft_query(devmode=True)
print(queryls)

total = 0

while (size == SIZE):
    print(f"Continue from record: {search_after}")
    data = OS_API.opensearch_request(size,queryls,search_after)
    df_os = pd.concat([df_os,data])
    
    #update params
    size = len(data) #exit once it completes last batch
    total+=size #update total records number
    search_after = data['sort_num'][len(data)-1] #update sort number to continue after
    #time.sleep(1)
    
print(f'Done. Total records: {total}')

[{'range': {'startTime': {'gte': 'now-1w/w', 'lte': 'now', 'format': 'strict_date_optional_time'}}}, {'exists': {'field': 'hashes'}}, {'match': {'geoip.country_name': 'Singapore'}}, {'query_string': {'default_field': 'commands', 'query': 'wget OR tftp'}}]
Continue from record: [0]
<Response [200]>, page successfully pulled.
Done. Total records: 2138


In [7]:
df_os

,Src IP,Src Port,Dst IP,Dst Port,commands,hashes,urls,loggedin,startTime,endTime,sort_num,Src Country,Dst Country
0,45.64.130.149,41676,103.153.254.12,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx]",2022-09-26T10:14:35.293683Z,2022-09-26T10:15:39.376497Z,[1664187339445],SG,NaN
1,45.64.130.149,48534,103.153.254.35,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx]",2022-09-26T10:14:35.111423Z,2022-09-26T10:15:39.533016Z,[1664187339600],SG,NaN
2,45.64.130.149,52868,103.153.254.35,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx#EDC]",2022-09-26T10:14:41.729100Z,2022-09-26T10:15:48.295857Z,[1664187348383],SG,NaN
3,45.64.130.149,46010,103.153.254.12,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx#EDC]",2022-09-26T10:14:41.773805Z,2022-09-26T10:15:49.215799Z,[1664187349352],SG,NaN
4,45.64.130.149,50350,103.153.254.12,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, @huawei123]",2022-09-26T10:14:50.106574Z,2022-09-26T10:15:53.727116Z,[1664187353792],SG,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,45.64.130.149,43996,104.145.227.232,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[a2f3d6d2bd82a65939f4e939bce242e8e246014fb3a9a...,[],"[root, Admin@hdfgj29]",2022-10-02T07:09:46.513562Z,2022-10-02T07:09:50.375706Z,[1664694590676],SG,US
2134,45.64.130.149,39476,104.145.239.235,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, Admin@hdfgj28]",2022-10-02T07:09:43.606422Z,2022-10-02T07:09:50.380457Z,[1664694590676],SG,CA
2135,45.64.130.149,41854,104.145.239.235,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,[http://sos.vivi.sg/ns1.jpg],"[root, Admin@hdfgj29]",2022-10-02T07:09:46.671710Z,2022-10-02T07:09:50.385487Z,[1664694590676],SG,CA
2136,45.64.130.149,60714,104.145.239.235,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, Admin@hdfgj25]",2022-10-02T07:09:33.960510Z,2022-10-02T07:09:50.365363Z,[1664694590676],SG,CA


## Searching Hashes 

In [8]:
hashls = []

for i in df_os['hashes']:
    for j in i:
        if j not in hashls:
            hashls.append(j)

hashls

['f494991106ba8bcca0641b6cc94e5423d5479ac87ef27d9dba5e9b827acd7212',
 'ae9e5007de8b71cca7a456d1913be332258c60b1fb1a444e8144d99251a144c7',
 'a2f3d6d2bd82a65939f4e939bce242e8e246014fb3a9a9d5c3769ed7dcfffe24']

In [9]:
resultls = []

for hsh in tqdm(hashls):
    response = VT_API.api_call_virustotal(hsh)
    #print(response.text)
    result = VT_API.read_response(response)
    result['hash'] = hsh
    resultls.append(result)
    time.sleep(6)

resultls
    

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:28<00:00,  9.59s/it]


[{'stats_output': 0.6271186440677966,
  'engines_flagged': ['MicroWorld-eScan',
   'ClamAV',
   'FireEye',
   'CAT-QuickHeal',
   'ALYac',
   'Sangfor',
   'K7AntiVirus',
   'K7GW',
   'Arcabit',
   'Baidu',
   'Cyren',
   'Symantec',
   'ESET-NOD32',
   'TrendMicro-HouseCall',
   'Avast',
   'Cynet',
   'Kaspersky',
   'BitDefender',
   'NANO-Antivirus',
   'Ad-Aware',
   'Emsisoft',
   'DrWeb',
   'TrendMicro',
   'McAfee-GW-Edition',
   'Sophos',
   'Ikarus',
   'Avira',
   'Microsoft',
   'GData',
   'AhnLab-V3',
   'McAfee',
   'Rising',
   'Yandex',
   'MAX',
   'Fortinet',
   'AVG',
   'Panda'],
  'harmless': 0,
  'malicious': 37,
  'suspicious': 0,
  'undetected': 22,
  'timeout': 0,
  'hash': 'f494991106ba8bcca0641b6cc94e5423d5479ac87ef27d9dba5e9b827acd7212'},
 {'stats_output': 0.017543859649122806,
  'engines_flagged': ['DrWeb'],
  'harmless': 0,
  'malicious': 1,
  'suspicious': 0,
  'undetected': 54,
  'timeout': 2,
  'hash': 'ae9e5007de8b71cca7a456d1913be332258c60b1fb1a444

## Looking at Commands

In [11]:
commandls = []

for i,row in df_os.iterrows():
    if "http" in row['commands']:
        print(i,row['commands'])

In [ ]:
'''
#export to csv
df_os.to_csv("opensearch_matched" + "_" + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + ".csv", index=False)
'''

In [ ]:
'''
#Date inputs
year = "2022"
print('start month?')
m1 = input()
print('start day?')
d1 = input()
print('end month?')
m2 = input()
print('end day?')
d2 = input()

start_date = f"{year}-{str(100+int(m1))[-2:]}-{str(100+int(d1))[-2:]}T00:00:00.000Z"
end_date = f"{year}-{str(100+int(m2))[-2:]}-{str(100+int(d2))[-2:]}T00:00:00.000Z"
print(f"Start date: {start_date}\n End date: {end_date}")
'''